In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, hp, tpe, space_eval

from sklearn.model_selection import KFold, TimeSeriesSplit
import lightgbm as lgb
from time import time
from tqdm import tqdm_notebook

from xgboost import XGBClassifier
import os

import gc
import warnings
warnings.filterwarnings('ignore')

/Users/didi/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [2]:
output = pd.read_csv('../output/final_sub_v2.csv')
sub   = pd.read_csv('../output/final_sub_v2.csv')

In [3]:
NROWS = 2000
test = pd.read_csv('../temp/test_temp' + str(NROWS) + '.csv')
train = pd.read_csv('../temp/train_temp' + str(NROWS) + '.csv')

In [4]:
# test['card_str'] = test["card1"].apply(lambda x: str(x)) + "_" + test["card2"].apply(lambda x: str(x)) +\
#                 "_" + test["card3"].apply(lambda x: str(x)) + "_" + test["card4"].apply(lambda x: str(x)) +\
#                 "_" + test["card5"].apply(lambda x: str(x)) + "_" + test["card6"].apply(lambda x: str(x))

In [5]:
# test['card_addr1'] = test["card1"].apply(lambda x: str(x)) + "_" + test["card2"].apply(lambda x: str(x)) +\
#                 "_" + test["card3"].apply(lambda x: str(x)) + "_" + test["card4"].apply(lambda x: str(x)) +\
#                 "_" + test["card5"].apply(lambda x: str(x)) + "_" + test["card6"].apply(lambda x: str(x)) +\
#                 "_" + test["addr1"].apply(lambda x: str(x))

In [6]:
test['card_addr1_P_emaildomain'] = test["card1"].apply(lambda x: str(x)) + "_" + test["card2"].apply(lambda x: str(x)) +\
                "_" + test["card3"].apply(lambda x: str(x)) + "_" + test["card4"].apply(lambda x: str(x)) +\
                "_" + test["card5"].apply(lambda x: str(x)) + "_" + test["card6"].apply(lambda x: str(x)) +\
                "_" + test["addr1"].apply(lambda x: str(x)) + "_" + test["P_emaildomain"].apply(lambda x: str(x))

train['card_addr1_P_emaildomain'] = train["card1"].apply(lambda x: str(x)) + "_" + train["card2"].apply(lambda x: str(x)) +\
                "_" + train["card3"].apply(lambda x: str(x)) + "_" + train["card4"].apply(lambda x: str(x)) +\
                "_" + train["card5"].apply(lambda x: str(x)) + "_" + train["card6"].apply(lambda x: str(x)) +\
                "_" + train["addr1"].apply(lambda x: str(x)) + "_" + train["P_emaildomain"].apply(lambda x: str(x))

In [34]:
# test['TransactionAmt_card_addr1_P_emaildomain'] = test["TransactionAmt"].apply(lambda x: str(x)) + "_" +\
#                 test["card1"].apply(lambda x: str(x)) + "_" + test["card2"].apply(lambda x: str(x)) +\
#                 "_" + test["card3"].apply(lambda x: str(x)) + "_" + test["card4"].apply(lambda x: str(x)) +\
#                 "_" + test["card5"].apply(lambda x: str(x)) + "_" + test["card6"].apply(lambda x: str(x)) +\
#                 "_" + test["addr1"].apply(lambda x: str(x)) + "_" + test["P_emaildomain"].apply(lambda x: str(x))

# train['TransactionAmt_card_addr1_P_emaildomain'] = train["TransactionAmt"].apply(lambda x: str(x)) + "_" +\
#                 train["card1"].apply(lambda x: str(x)) + "_" + train["card2"].apply(lambda x: str(x)) +\
#                 "_" + train["card3"].apply(lambda x: str(x)) + "_" + train["card4"].apply(lambda x: str(x)) +\
#                 "_" + train["card5"].apply(lambda x: str(x)) + "_" + train["card6"].apply(lambda x: str(x)) +\
#                 "_" + train["addr1"].apply(lambda x: str(x)) + "_" + train["P_emaildomain"].apply(lambda x: str(x))

In [7]:
freatures = ["TransactionID", "isFraud", "card_addr1_P_emaildomain"]

In [8]:
train.loc[train["isFraud"] == 1, freatures]

,TransactionID,isFraud,card_addr1_P_emaildomain
203,2987203,1,18268_583.0_150.0_visa_226.0_credit_251.0_aol.com
240,2987240,1,13413_103.0_185.0_visa_137.0_credit_nan_hotmai...
243,2987243,1,13413_103.0_185.0_visa_137.0_credit_nan_hotmai...
245,2987245,1,13413_103.0_185.0_visa_137.0_credit_nan_hotmai...
288,2987288,1,16578_545.0_185.0_visa_226.0_credit_nan_outloo...
367,2987367,1,4425_562.0_150.0_mastercard_197.0_credit_472.0...
405,2987405,1,4504_500.0_185.0_mastercard_219.0_credit_nan_g...
630,2987630,1,5812_408.0_185.0_mastercard_224.0_debit_nan_gm...
683,2987683,1,5812_408.0_185.0_mastercard_224.0_debit_nan_gm...
736,2987736,1,15063_nan_150.0_visa_226.0_credit_231.0_yahoo.com


In [9]:
delta = 30
train["window_delta_" + str(delta)] = 0

train[freatures + ["window_delta_" + str(delta)]]

,TransactionID,isFraud,card_addr1_P_emaildomain,window_delta_30
0,2987000,0,13926_nan_150.0_discover_142.0_credit_315.0_nan,0
1,2987001,0,2755_404.0_150.0_mastercard_102.0_credit_325.0...,0
2,2987002,0,4663_490.0_150.0_visa_166.0_debit_330.0_outloo...,0
3,2987003,0,18132_567.0_150.0_mastercard_117.0_debit_476.0...,0
4,2987004,0,4497_514.0_150.0_mastercard_102.0_credit_420.0...,0
5,2987005,0,5937_555.0_150.0_visa_226.0_debit_272.0_gmail.com,0
6,2987006,0,12308_360.0_150.0_visa_166.0_debit_126.0_yahoo...,0
7,2987007,0,12695_490.0_150.0_visa_226.0_debit_325.0_mail.com,0
8,2987008,0,2803_100.0_150.0_visa_226.0_debit_337.0_anonym...,0
9,2987009,0,17399_111.0_150.0_mastercard_224.0_debit_204.0...,0


In [10]:
for i in tqdm_notebook(range(delta, len(train))):
    cur = i
    start = cur - delta
    end =   cur + delta
    cur_card_str = train.loc[cur, "card_addr1_P_emaildomain"]
    cnt  = train.loc[start:end].loc[train["card_addr1_P_emaildomain"] == cur_card_str].shape[0] - 1
    train.loc[cur, "window_delta_" + str(delta)] = cnt

In [12]:
a = train.loc[(train["isFraud"] == 1) & (train["window_delta_" + str(delta)] != 0), freatures + ["window_delta_" + str(delta)]].shape[0]
b = train.loc[(train["isFraud"] == 1), freatures + ["window_delta_" + str(delta)]].shape[0]
print(a,b)
print(a/b)

16 41
0.3902439024390244


In [20]:
a = train.loc[(train["isFraud"] == 0) & (train["window_delta_" + str(delta)] != 0), freatures + ["window_delta_" + str(delta)]].shape[0]
b = train.loc[(train["isFraud"] == 0), freatures + ["window_delta_" + str(delta)]].shape[0]
print(a,b)
print(a/b)

307 1959
0.15671260847371107


In [8]:
output = output.merge(test[["TransactionID", "TransactionAmt_card_addr1_P_emaildomain"]], on="TransactionID", how="left")


In [9]:
test[["TransactionID", "TransactionAmt_card_addr1_P_emaildomain"]].head()

,TransactionID,TransactionAmt_card_addr1_P_emaildomain
0,3663549,31.95_10409_111.0_150.0_visa_226.0_debit_170.0...
1,3663550,49.0_4272_111.0_150.0_visa_226.0_debit_299.0_a...
2,3663551,171.0_4476_574.0_150.0_visa_226.0_debit_472.0_...
3,3663552,284.95_10989_360.0_150.0_visa_166.0_debit_205....
4,3663553,67.95_18018_452.0_150.0_mastercard_117.0_debit...


In [10]:
output.head()

,TransactionID,isFraud,TransactionAmt_card_addr1_P_emaildomain
0,3663549,0.038136,31.95_10409_111.0_150.0_visa_226.0_debit_170.0...
1,3663550,0.072050,49.0_4272_111.0_150.0_visa_226.0_debit_299.0_a...
2,3663551,0.069657,171.0_4476_574.0_150.0_visa_226.0_debit_472.0_...
3,3663552,0.091089,284.95_10989_360.0_150.0_visa_166.0_debit_205....
4,3663553,0.042804,67.95_18018_452.0_150.0_mastercard_117.0_debit...


In [11]:
output.loc[output["isFraud"] > 0.9]

,TransactionID,isFraud,TransactionAmt_card_addr1_P_emaildomain
220,3663769,0.901452,6.148_3154_408.0_185.0_mastercard_224.0_debit_...
278,3663827,0.988671,250.0_16727_111.0_150.0_visa_226.0_credit_441....
279,3663828,0.981312,250.0_16727_111.0_150.0_visa_226.0_credit_441....
287,3663836,0.995031,250.0_11106_100.0_150.0_visa_226.0_credit_337....
554,3664103,0.906737,9.529_3154_408.0_185.0_mastercard_224.0_debit_...
1338,3664887,0.966060,225.0_16659_170.0_150.0_visa_226.0_credit_177....
1359,3664908,0.977771,10.206_10086_500.0_185.0_mastercard_224.0_cred...
1364,3664913,0.977583,10.206_4606_141.0_185.0_visa_137.0_credit_nan_...
1372,3664921,0.983032,10.206_3154_408.0_185.0_mastercard_224.0_debit...
1443,3664992,0.911939,42.974_9633_130.0_185.0_visa_138.0_debit_nan_h...


In [11]:
int(output.shape[0]*0.2)

101338

In [12]:
output = output.loc[:101340]

In [13]:
output.shape

(101341, 3)

In [39]:
# rule

rule = output.loc[output["isFraud"] > 0.95]
rule.index = range(len(rule))

delta = 3
rule["window_delta_" + str(delta)] = 0

rule[["TransactionID", "isFraud", "card_str", "window_delta_" + str(delta)]]


for i in tqdm_notebook(range(delta, len(rule))):
    cur = i
    start = cur - delta
    end =   cur + delta
    cur_card_str = rule.loc[cur, "card_str"]
    cnt  = rule.loc[start:end].loc[rule["card_str"] == cur_card_str].shape[0] - 1
    rule.loc[cur, "window_delta_" + str(delta)] = cnt
    
    
rule[["TransactionID", "isFraud", "card_str", "window_delta_" + str(delta)]]


sub.loc[sub["TransactionID"].isin(list(rule.loc[rule["window_delta_3"] != 0, "TransactionID"])), "isFraud"] = 1.


sub.loc[sub["TransactionID"].isin(list(rule.loc[rule["window_delta_3"] != 0, "TransactionID"]))]


sub.to_csv('../output/final_sub_v2_rule1.csv', index=False)

In [17]:
TransactionID = 3764849

sub.loc[sub['TransactionID'] == TransactionID, "isFraud"]

101300    0.966982
Name: isFraud, dtype: float64

In [18]:
sub.loc[sub['TransactionID'] == TransactionID, "isFraud"] = 0.

In [19]:
sub.loc[sub['TransactionID'] == TransactionID, "isFraud"]

101300    0.0
Name: isFraud, dtype: float64

In [20]:
sub.shape

(506691, 2)

In [21]:
sub.to_csv('../output/final_sub_v2_' + str(TransactionID) + '.csv', index=False)

In [ ]:
# label为1的样本
3697927
3764849

# label为0的样本
3668193